# part 1 get bed files

In [3]:
#part1.1:  get position bed from transcript bed +
# v0.2 add stop codon
def getPosBed_Plus(transBed):
    arrTest=transBed
    
    ## init paras
    arrExon=[]
    arrUTR=[]
    start_codon=[] #one
    stop_codon=[] #one
    #
    for tmp in arrTest:
        #arr3=arrTest[i];
        if tmp[-1]=='exon':
            arrExon.append(tmp)
        elif tmp[-1]=='UTR':
            arrUTR.append(tmp)
        elif tmp[-1]=='start_codon':
            start_codon=tmp;
        elif tmp[-1]=='stop_codon':
            stop_codon=tmp;
        #print(arrTest[i])
    #print(len(arrExon), len(arrUTR))
    
    ## maybe have 3UTR
    UTR3=[]
    UTR5=[]
    if len(arrUTR)>0:
        for tmp in arrUTR:
            if len(start_codon)>0 and tmp[2]<start_codon[1]:
                tmp[-1]="UTR5"
                UTR5.append(tmp)
            elif len(stop_codon)>0 and tmp[1]>=stop_codon[1]:
                tmp[-1]="UTR3"
                UTR3.append(tmp)
    #print(len(UTR5), len(UTR3))
    
    ## maybe have e3UTR
    eUTR3=[]
    if len(UTR3)>0:
        eUTR3=UTR3[-1].copy()
        #
        eUTR3[1]= str( int(eUTR3[2])+1 )
        eUTR3[2]= str( int(eUTR3[2])+1000 )
        eUTR3[6]='extended3UTR'
        #print(UTR3[-1])
        #print(eUTR3)
    ##get arrIntron，maybe
    arrIntron=[]
    #print(arrTest[0], '\n')
    intron=[]
    if len(arrExon)>1: #if only one exon, then no intron.
        for i in range(len(arrExon)):
            tmp=arrExon[i]
            if len(intron)==0: #first exon
                intron=tmp.copy()
                intron[1]= str( int(intron[2])+1 )
                continue;
            else:
                #2nd
                intron[2]=str( int(tmp[1])-1 )
                intron[-1]="intron"
                arrIntron.append(intron);
                #
                intron=tmp.copy();
                intron[1]= str( int(intron[2])+1 )
    #
    #print('arrExon=',arrExon)
    #print('arrIntron=',arrIntron)

    ##get TSS，must have
    TSS=arrTest[0].copy()
    pos=int(arrTest[0][1])
    TSS[2]=str( pos+10 )
    TSS[1]=str( pos-10 )
    TSS[-1]="TSS"
    #print(arrTest[0])
    #TSS
    
    ##get Promoter，must have
    Promoter=arrTest[0].copy()
    Promoter[1]=str( pos-250 )
    Promoter[2]=str( pos-11 )
    Promoter[-1]='Promoter'
    #print(arrTest[0])
    #Promoter
    
    #get PA, must have
    PA=arrTest[0].copy()
    pos2=int(arrTest[0][2])
    PA[1]=str( pos2-10 )
    PA[2]=str( pos2+10 )
    PA[-1]='PA'
    
    # combine
    saveSet=[]
    saveSet.extend(UTR3) #many
    saveSet.append(eUTR3) #1

    saveSet.extend(arrExon) #many
    saveSet.extend(arrIntron) #many
    saveSet.extend(UTR5) #many

    saveSet.append(TSS) #1
    saveSet.append(Promoter) #1
    
    saveSet.append(arrTest[0]) #1
    
    saveSet.append(stop_codon)
    saveSet.append(start_codon)
    saveSet.append(PA)

    return saveSet

In [4]:
#test
arrTestP=[
    ['chr1', '65419', '71585', 'OR4F5-202', '0', '+', 'transcript'],
    ['chr1', '65419', '65433', 'OR4F5-202', '0', '+', 'exon'],
    ['chr1', '65520', '65573', 'OR4F5-202', '0', '+', 'exon'],
    ['chr1', '65565', '65567', 'OR4F5-202', '0', '+', 'start_codon'],
    ['chr1', '69037', '71585', 'OR4F5-202', '0', '+', 'exon'],
    ['chr1', '70006', '70008', 'OR4F5-202', '0', '+', 'stop_codon'],
    ['chr1', '65419', '65433', 'OR4F5-202', '0', '+', 'UTR'],
    ['chr1', '65520', '65564', 'OR4F5-202', '0', '+', 'UTR'],
    ['chr1', '70006', '71585', 'OR4F5-202', '0', '+', 'UTR']
]
getPosBed_Plus(arrTestP)

[['chr1', '70006', '71585', 'OR4F5-202', '0', '+', 'UTR3'],
 ['chr1', '71586', '72585', 'OR4F5-202', '0', '+', 'extended3UTR'],
 ['chr1', '65419', '65433', 'OR4F5-202', '0', '+', 'exon'],
 ['chr1', '65520', '65573', 'OR4F5-202', '0', '+', 'exon'],
 ['chr1', '69037', '71585', 'OR4F5-202', '0', '+', 'exon'],
 ['chr1', '65434', '65519', 'OR4F5-202', '0', '+', 'intron'],
 ['chr1', '65574', '69036', 'OR4F5-202', '0', '+', 'intron'],
 ['chr1', '65419', '65433', 'OR4F5-202', '0', '+', 'UTR5'],
 ['chr1', '65520', '65564', 'OR4F5-202', '0', '+', 'UTR5'],
 ['chr1', '65409', '65429', 'OR4F5-202', '0', '+', 'TSS'],
 ['chr1', '65169', '65408', 'OR4F5-202', '0', '+', 'Promoter'],
 ['chr1', '65419', '71585', 'OR4F5-202', '0', '+', 'transcript'],
 ['chr1', '70006', '70008', 'OR4F5-202', '0', '+', 'stop_codon'],
 ['chr1', '65565', '65567', 'OR4F5-202', '0', '+', 'start_codon'],
 ['chr1', '71575', '71595', 'OR4F5-202', '0', '+', 'PA']]

In [5]:
#part1.2: get position bed from transcript bed -
# v0.2 add stop codon
def getPosBed_Minus(transBed):
    arrTest=transBed
    
    ## init paras
    arrExon=[]
    arrUTR=[]
    start_codon=[] #one
    stop_codon=[] #one
    #
    for tmp in arrTest:
        #arr3=arrTest[i];
        if tmp[-1]=='exon':
            arrExon.append(tmp)
        elif tmp[-1]=='UTR':
            arrUTR.append(tmp)
        elif tmp[-1]=='start_codon':
            start_codon=tmp;
        elif tmp[-1]=='stop_codon':
            stop_codon=tmp;
        #print(arrTest[i])
    #print(len(arrExon), len(arrUTR))

    
    ## get 3UTR, maybe
    UTR3=[]
    UTR5=[]
    if len(arrUTR)>0:
        for tmp in arrUTR:
            if len(start_codon)>0 and tmp[1]>start_codon[2]:
                tmp[-1]="UTR5"
                UTR5.append(tmp)
            elif len(stop_codon)>0 and tmp[2]<=stop_codon[2]:
                tmp[-1]="UTR3"
                UTR3.append(tmp)
    #print(len(UTR5), len(UTR3))
    
    
    ## get e3UTR, maybe
    eUTR3=[]
    if len(UTR3)>0:
        eUTR3=UTR3[-1].copy()
        #
        eUTR3[2]= str( int(eUTR3[1])-1 )
        eUTR3[1]= str( int(eUTR3[1])-1000 )
        eUTR3[6]='extended3UTR'
        #print(UTR3[-1])
        #print(eUTR3)
        
    ##get arrIntron，must
    arrIntron=[]
    #print(arrTest[0], '\n')
    intron=[]
    if len(arrExon)>1:
        for i in range(len(arrExon)):
            tmp=arrExon[i]
            if len(intron)==0: #first exon
                intron=tmp.copy()
                intron[2]= str( int(intron[1])-1 )
                continue;
            else:
                #第二个时
                intron[1]=str( int(tmp[2])+1 )
                intron[-1]="intron"
                arrIntron.append(intron);
                #
                intron=tmp.copy();
                intron[2]= str( int(intron[1])-1 )
    #
    #print('arrExon=',arrExon)
    #print('arrIntron=',arrIntron)

    ##get TSS，must
    TSS=arrTest[0].copy()
    pos=int(arrTest[0][2])
    TSS[2]=str( pos+10 )
    TSS[1]=str( pos-10 )
    TSS[-1]="TSS"
    #print(arrTest[0])
    #TSS
    
    ##get Promoter，must
    Promoter=arrTest[0].copy()
    Promoter[1]=str( pos+11 )
    Promoter[2]=str( pos+250 )
    Promoter[-1]='Promoter'
    #print(arrTest[0])
    #Promoter
    
    #get PA must
    PA=arrTest[0].copy()
    pos2=int(arrTest[0][1])
    PA[1]=str( pos2-10 )
    PA[2]=str( pos2+10 )
    PA[-1]='PA'
    
    # 合并
    saveSet=[]
    saveSet.extend(UTR3) #many
    saveSet.append(eUTR3) #1

    saveSet.extend(arrExon) #many
    saveSet.extend(arrIntron) #many
    saveSet.extend(UTR5) #many

    saveSet.append(TSS) #1
    saveSet.append(Promoter) #1
    saveSet.append(arrTest[0]) #1
    
    saveSet.append(stop_codon)
    saveSet.append(start_codon)
    
    saveSet.append(PA)

    return saveSet

In [6]:
#test
arrTestM=[
    ['chr1', '879583', '894636', 'NOC2L-201', '0', '-', 'transcript'],
    ['chr1', '894595', '894636', 'NOC2L-201', '0', '-', 'exon'],
    ['chr1', '894618', '894620', 'NOC2L-201', '0', '-', 'start_codon'],
    ['chr1', '894309', '894461', 'NOC2L-201', '0', '-', 'exon'],
    ['chr1', '892479', '892653', 'NOC2L-201', '0', '-', 'exon'],
    ['chr1', '892274', '892405', 'NOC2L-201', '0', '-', 'exon'],
    ['chr1', '891475', '891595', 'NOC2L-201', '0', '-', 'exon'],
    ['chr1', '891303', '891393', 'NOC2L-201', '0', '-', 'exon'],
    ['chr1', '889384', '889462', 'NOC2L-201', '0', '-', 'exon'],
    ['chr1', '889162', '889272', 'NOC2L-201', '0', '-', 'exon'],
    ['chr1', '888555', '888668', 'NOC2L-201', '0', '-', 'exon'],
    ['chr1', '887792', '887980', 'NOC2L-201', '0', '-', 'exon'],
    ['chr1', '887380', '887519', 'NOC2L-201', '0', '-', 'exon'],
    ['chr1', '886507', '886618', 'NOC2L-201', '0', '-', 'exon'],
    ['chr1', '883870', '883983', 'NOC2L-201', '0', '-', 'exon'],
    ['chr1', '883511', '883612', 'NOC2L-201', '0', '-', 'exon'],
    ['chr1', '881782', '881925', 'NOC2L-201', '0', '-', 'exon'],
    ['chr1', '881553', '881666', 'NOC2L-201', '0', '-', 'exon'],
    ['chr1', '880898', '881033', 'NOC2L-201', '0', '-', 'exon'],
    ['chr1', '880437', '880526', 'NOC2L-201', '0', '-', 'exon'],
    ['chr1', '879583', '880180', 'NOC2L-201', '0', '-', 'exon'],
    ['chr1', '880074', '880076', 'NOC2L-201', '0', '-', 'stop_codon'],
    ['chr1', '894621', '894636', 'NOC2L-201', '0', '-', 'UTR'],
    ['chr1', '879583', '880076', 'NOC2L-201', '0', '-', 'UTR']
]
getPosBed_Minus(arrTestM)

[['chr1', '879583', '880076', 'NOC2L-201', '0', '-', 'UTR3'],
 ['chr1', '878583', '879582', 'NOC2L-201', '0', '-', 'extended3UTR'],
 ['chr1', '894595', '894636', 'NOC2L-201', '0', '-', 'exon'],
 ['chr1', '894309', '894461', 'NOC2L-201', '0', '-', 'exon'],
 ['chr1', '892479', '892653', 'NOC2L-201', '0', '-', 'exon'],
 ['chr1', '892274', '892405', 'NOC2L-201', '0', '-', 'exon'],
 ['chr1', '891475', '891595', 'NOC2L-201', '0', '-', 'exon'],
 ['chr1', '891303', '891393', 'NOC2L-201', '0', '-', 'exon'],
 ['chr1', '889384', '889462', 'NOC2L-201', '0', '-', 'exon'],
 ['chr1', '889162', '889272', 'NOC2L-201', '0', '-', 'exon'],
 ['chr1', '888555', '888668', 'NOC2L-201', '0', '-', 'exon'],
 ['chr1', '887792', '887980', 'NOC2L-201', '0', '-', 'exon'],
 ['chr1', '887380', '887519', 'NOC2L-201', '0', '-', 'exon'],
 ['chr1', '886507', '886618', 'NOC2L-201', '0', '-', 'exon'],
 ['chr1', '883870', '883983', 'NOC2L-201', '0', '-', 'exon'],
 ['chr1', '883511', '883612', 'NOC2L-201', '0', '-', 'exon'],


In [7]:
#genomic location based on GTF
import re, time

gtf='/home/wangjl/data/ref/hg19/gencode.v30lift37.annotation.gtf'
fr=open(gtf,'r',encoding="utf-8")

start_time=time.time()

arrFile={}
dirPath_beds="/home/wangjl/data/apa/191111Figure/f2/positionPY/"
arrFile['UTR3']=open(dirPath_beds+"UTR3.bed", 'w')
arrFile['extended3UTR']=open(dirPath_beds+"extended3UTR.bed", 'w')
arrFile['exon']=open(dirPath_beds+"exon.bed", 'w')
arrFile['intron']=open(dirPath_beds+"intron.bed", 'w')
arrFile['UTR5']=open(dirPath_beds+"UTR5.bed", 'w')
arrFile['TSS']=open(dirPath_beds+"TSS.bed", 'w')
arrFile['Promoter']=open(dirPath_beds+"Promoter.bed", 'w')
arrFile['transcript']=open(dirPath_beds+"transcript.bed", 'w')
#
arrFile['start_codon']=open(dirPath_beds+"start_codon.bed", 'w')
arrFile['stop_codon']=open(dirPath_beds+"stop_codon.bed", 'w')
arrFile['PA']=open(dirPath_beds+"PA.bed", 'w')

#
i=0
db=[]
totalLine=0
while True:
    totalLine+=1
    line=fr.readline()
    if not line: #last line
        break
    
    line=line.strip()
    
    # filter: not in use now
    if not re.search('protein_coding', line):
        #continue;
        pass
    
    # filter: discard comment lines #
    if line.startswith("#"):
        continue;

    #
    arr=re.split('\t', line) ##['chr1', 'HAVANA', 'transcript', '65419', '71585', '.', '+', '.', ...]    
    # filter out cds
    if arr[2]=='CDS' or arr[2]=='gene':
        continue;
    #
        
    #for debug
    i+=1;
    if i>3000:
        #break;
        pass
    
    if i%10000==0:
        print(i, round(time.time()-start_time,2), 'second elapsed.')
    
    ##############
    # get a transcript
    arr2=re.split('; ', arr[8])
    #print( arr2 )
    #['gene_id "ENSG00000186092.6_4"', 'transcript_id "ENST00000641515.2_2"', 
    #'gene_type "protein_coding"', 'gene_name "OR4F5"', 'transcript_type "protein_coding"', 
    #'transcript_name "OR4F5-202"', 'level 2', 'protein_id "ENSP00000493376.2"', 'tag "RNA_Seq_supported_partial"', 
    #'tag "basic"', 'havana_gene "OTTHUMG00000001094.4_4"', 'havana_transcript "OTTHUMT00000003223.4_2"', 'remap_num_mappings 1', 
    #'remap_status "full_contig"', 'remap_target_status "new";']

    dictG={}
    for item in arr2:
        (k,v)=re.split(' ',item)
        dictG[k]=v.strip('"')
    #rint('dictG=',dictG)
    if 'transcript_name' in dictG:
        transcript_name=dictG['transcript_name']
    else:
        transcript_name=""
        #continue;
    #
    arrN=[arr[0], arr[3], arr[4], transcript_name, "0", arr[6], arr[2] ]
        
    #save to file, according to type(start, end, 3UTR, e3UTR, exon, intron, 5UTR, TSS, promoter) and strand.
    if arr[2]=='transcript':
        if len(db)>0:
            #print('i=', i, '; len(db)=', len(db))            
            #for j in db:
            #    print(j) # ['chr1', '65419', '71585', 'OR4F5-202', '0', '+']
            #    pass
            
            #get strand
            if db[0][5]=='-':
                saveSet=getPosBed_Minus(db)
            else:
                saveSet=getPosBed_Plus(db)
            #save to file
            #print('; len(saveSet)=',len(saveSet))
            for j in saveSet:
                if len(j)>0:
                    arrFile[j[-1]].write( '\t'.join(j)+'\n' ) # ['chr1', '65419', '71585', 'OR4F5-202', '0', '+']
                #pass
            
        
        # clear and next transcript
        db=[]
        db.append(arrN)
        #print('=*'*20)
    else:
        db.append(arrN)
    
    #print(i, arr[0:8], len(arr))
    
fr.close()

for f in arrFile:
    arrFile[f].close()
print('==main end== i=',i, '; totalLine=',totalLine) #57s 2782645 lines

10000 0.37 second elapsed.
20000 0.66 second elapsed.
30000 0.95 second elapsed.
40000 1.24 second elapsed.
50000 1.53 second elapsed.
60000 1.81 second elapsed.
70000 2.1 second elapsed.
80000 2.39 second elapsed.
90000 2.67 second elapsed.
100000 2.96 second elapsed.
110000 3.25 second elapsed.
120000 3.54 second elapsed.
130000 3.83 second elapsed.
140000 4.11 second elapsed.
150000 4.41 second elapsed.
160000 4.69 second elapsed.
170000 4.99 second elapsed.
180000 5.27 second elapsed.
190000 5.56 second elapsed.
200000 5.85 second elapsed.
210000 6.14 second elapsed.
220000 6.43 second elapsed.
230000 6.72 second elapsed.
240000 7.0 second elapsed.
250000 7.29 second elapsed.
260000 7.58 second elapsed.
270000 7.88 second elapsed.
280000 8.17 second elapsed.
290000 8.46 second elapsed.
300000 8.75 second elapsed.
310000 9.05 second elapsed.
320000 9.51 second elapsed.
330000 9.79 second elapsed.
340000 10.08 second elapsed.
350000 10.37 second elapsed.
360000 10.66 second elapsed.


# part 2 read bed files to memory

In [50]:
# position (as para name)= chr:strand { 12: gene1, 15: gene2} 

import re,time
def readBed(bedName, verbose=False):
    print("File name: ",bedName)
    start_time=time.time()
    fr=open("/home/wangjl/data/apa/191111Figure/f2/positionPY/"+bedName+".bed", 'r')

    i=0
    mem={}
    for lineR in fr.readlines():
        i+=1
        if i>1000:
            #break;
            pass
        if i%1e5==0:
            print('\t==> ',i, round(time.time()-start_time,2), 'second elapsed.')
        
        line=lineR.strip()
        arr=re.split('\t', line)
        #print(i, arr)

        chrStrand=arr[0]+":"+arr[5]
        if chrStrand not in mem:
            mem[chrStrand]={}
        for j in range( int(arr[1]), int(arr[2])+1):
            mem[chrStrand][j]=arr[3]

    fr.close()
    print('\t==end==, i=', i, bedName)
    if verbose:
        for chrStrand in mem:
            print(chrStrand, len( mem[chrStrand]) )
    return mem;

#test
#extended3UTR=readBed("extended3UTR", True)

# part 3New(use less memory) annotate location and transcript name for each given point

In [47]:
# add 2 cols
posFile="/data/jinwf/wangjl/apa/190705PAS/bed/pasPostions_siteAndIncludes_PY.bed"
outputFile="/data/jinwf/wangjl/apa/190705PAS/bed/pasPostions_Location_transcriptName_PY.bed"

def extend2Column():
    fr=open(posFile,'r',encoding="utf-8")
    fw=open(outputFile, 'w')

    i=0
    for lineR in fr.readlines():
        i+=1
        line=lineR.strip()
        arr=re.split('\t',line)
        # if only 6 cols, then add 2 last cols
        if len(arr)<=6:
            arr.append('intergenic') #region
            arr.append('.') #gene names
        fw.write('\t'.join(arr)+"\n")    
    fr.close();
    fw.close();
    print('==end==,i=',i);

extend2Column();

==end==,i= 742288


In [ ]:
#
# update position genomic region
def updateRegion(location, outputFile):
    # read the DB;
    posDB=readBed(location)
    
    #middle file;
    tmpFile="tmp_123456789.txt"

    fr=open(outputFile, 'r')
    fw=open(tmpFile, 'w')
    i=0;
    j=0;
    for lineR in fr.readlines():
        i+=1;
        if i>100:
            #break;
            pass;
        line=lineR.strip()
        arr=re.split('\t',line)

        # jump if have annotated, so first annotate prioritized regions.
        if arr[6]!='intergenic':
            fw.write('\t'.join(arr)+"\n");
            continue;

        # add annotate info, refer to PAS DB
        chrStrand=arr[0]+':'+arr[5]
        pos=int(arr[1])
        if arr[5]=="-":
            pos+=1
        if chrStrand in posDB:
            if pos in posDB[chrStrand]:
                transcriptName=posDB[chrStrand][pos]
                # if find, over, and write down
                arr[6]=location;
                arr[7]=transcriptName
                j+=1;
        fw.write('\t'.join(arr)+"\n");
    
    fr.close();
    fw.close();
    print(location, 'change=',j, ', total=',i, round(time.time()- start_time, 2),'seconds elapsed.')

    #change file name
    os.rename(tmpFile, outputFile);
#

In [51]:
# v0.2
#task1: OK;
# depend on readBed() before;

import re, time, os
# location, transcript name;


def main():
    #1. add 2 cols
    extend2Column();
    
    #2. add annotation
    locatonArr=['PA', 'UTR3','extended3UTR', 'exon','intron','UTR5','TSS','Promoter']
    for location in locatonArr:
        t1=time.time();
        print('='*20, location)
        updateRegion(location, outputFile)
        print('-'*20, round(time.time()-t1,2),'s' )
#

start_time=time.time();
main();
print('==end==');

==end==,i= 742288
==================== PA
File name:  PA
	==>  100000 0.79 second elapsed.
	==>  200000 1.31 second elapsed.
	==end==, i= 210424 PA
PA change= 27726 , total= 742288 4.82 seconds elapsed.
-------------------- 3.59 s
==================== UTR3
File name:  UTR3
	==>  100000 9.48 second elapsed.
	==end==, i= 154332 UTR3
UTR3 change= 73214 , total= 742288 20.43 seconds elapsed.
-------------------- 16.25 s
==================== extended3UTR
File name:  extended3UTR
	==end==, i= 78919 extended3UTR
extended3UTR change= 13083 , total= 742288 34.43 seconds elapsed.
-------------------- 14.03 s
==================== exon
File name:  exon
	==>  100000 4.23 second elapsed.
	==>  200000 8.46 second elapsed.
	==>  300000 12.02 second elapsed.
	==>  400000 18.41 second elapsed.
	==>  500000 22.35 second elapsed.
	==>  600000 26.34 second elapsed.
	==>  700000 30.46 second elapsed.
	==>  800000 34.06 second elapsed.
	==>  900000 37.83 second elapsed.
	==>  1000000 41.53 second elapsed.
	=

## for DB3

In [57]:
# for DB3

# add cols
posFile=   "/home/wangjl/data/apa/190610APA/02_validate/human.PAS.txt"
outputFile="/data/jinwf/wangjl/apa/190705PAS/bed/PolyA_DB3_location_transcriptName.bed"
#chr1    564599  564599  chr1:564599:+   0       +       exon    MTND1P23-201


def extend2Column2(posFile2, outputFile2):
    fr=open(posFile2,'r',encoding="utf-8")
    fw=open(outputFile2, 'w')

    i=0
    for lineR in fr.readlines():
        i+=1
        if i==1:
            continue;
        if i>10:
            #break;
            pass;
    
        line=lineR.strip()
        arr=re.split('\t',line)
        
        arrN=[arr[1], arr[2],arr[2],arr[0], "0", arr[3]]
        ##chr10	157706	157706	chr10:157706:-	5	-
        
        # add 2 cols if only 6 cols.
        if len(arrN)<=6:
            arrN.append('intergenic') #regions
            arrN.append('.') #gene symbols
        fw.write('\t'.join(arrN)+"\n")    
    fr.close();
    fw.close();
    print('==end==,i=',i);

extend2Column2(posFile, outputFile);

==end==,i= 311595


In [58]:
#######################
# ===> annotate site for polyA_DB3
#######################
def main2():
    #1. add cols
    extend2Column2(posFile, outputFile);
    
    #2. add annotation
    locatonArr=['PA', 'UTR3','extended3UTR', 'exon','intron','UTR5','TSS','Promoter']
    for location in locatonArr:
        t1=time.time();
        print('='*20, location)
        updateRegion(location, outputFile)
        print('-'*20, round(time.time()-t1,2),'s' )
#

start_time=time.time();
main2();
print('==end==');

==end==,i= 311595
==================== PA
File name:  PA
	==>  100000 0.52 second elapsed.
	==>  200000 1.03 second elapsed.
	==end==, i= 210424 PA
PA change= 36015 , total= 311594 2.74 seconds elapsed.
-------------------- 1.91 s
==================== UTR3
File name:  UTR3
	==>  100000 8.34 second elapsed.
	==end==, i= 154332 UTR3
UTR3 change= 74951 , total= 311594 15.85 seconds elapsed.
-------------------- 13.74 s
==================== extended3UTR
File name:  extended3UTR
	==end==, i= 78919 extended3UTR
extended3UTR change= 14991 , total= 311594 28.43 seconds elapsed.
-------------------- 12.49 s
==================== exon
File name:  exon
	==>  100000 4.38 second elapsed.
	==>  200000 8.68 second elapsed.
	==>  300000 12.25 second elapsed.
	==>  400000 16.6 second elapsed.
	==>  500000 20.56 second elapsed.
	==>  600000 24.55 second elapsed.
	==>  700000 28.24 second elapsed.
	==>  800000 31.9 second elapsed.
	==>  900000 35.69 second elapsed.
	==>  1000000 39.4 second elapsed.
	==> 